### 🔢 Zadanie 1: Oblicz predykcję

Masz podane dane:

- x np. [1, 5]  → [czas nauki = 1h, nieobecności = 5]
- wagi np. β₀ = -4, β₁ = 2, β₂ = -1

**Oblicz wynik modelu (czyli wartość funkcji sigmoidalnej)**:

1. Najpierw oblicz z = β₀ + β₁·x₁ + β₂·x₂
2. Potem p = 1 / (1 + exp(-z))


In [ ]:
import math

def calculate_prediction(X, weights):
  sum = weights[0]

  for i in range(len(X)):
    sum += weights[i+1] * X[i]

  return 1 / (1 + math.exp(-sum))
  # Your implementation here
  pass

In [ ]:
def test_calculate_prediction():
  assert (calculate_prediction([1, 5], [4, 2, 1]) > 0.5)
  assert (calculate_prediction([1, 5], [-4, 2, -1]) > 0.5) == False
  print("OK")

test_calculate_prediction()


0.999983298578152
OK


### 🔢 Zadanie 2: Oblicz predykcję wersja HARD!!1

Dokładnie to samo ale `x` oraz `weights` mogą mieć różne rozmiary (oczywiście `len(x)+1 == len(weights)`)

**Oblicz wynik modelu (czyli wartość funkcji sigmoidalnej)**:


In [ ]:
import math

def calculate_prediction_hard(X, weights):
  sum = weights[0]

  for i in range(len(X)):
    sum += weights[i+1] * X[i]

  return 1 / (1 + math.exp(-sum))
  # Your implementation here
  pass

In [ ]:
def test_calculate_prediction_hard():
  assert (calculate_prediction_hard([1, 5], [4, 2, 1]) > 0.5)
  assert (calculate_prediction_hard([1, 5], [-4, 2, -1]) > 0.5) == False
  assert (calculate_prediction_hard([1, 5, 10], [4, 2, 1, 1]) > 0.5)
  assert (calculate_prediction_hard([1, 5, 10], [-4, 2, -1, 1]) > 0.5)
  assert (calculate_prediction_hard([1, 5, 10, -10, -4, -2], [4, 2, 1, 1, 4, -1, -2]) > 0.5) == False

  print("OK")

test_calculate_prediction_hard()


OK


## 🚢 Zbiór danych: Titanic – klasyfikacja przeżycia
📄 Opis:
`Pclass`: Klasa biletu (1 = 1st, 2 = 2nd, 3 = 3rd)

`Sex`: Płeć pasażera

`Age`: Wiek w latach

`SibSp`: Liczba rodzeństwa / małżonków na pokładzie

`Parch`: Liczba rodziców / dzieci na pokładzie

`Fare`: Cena biletu

`Survived`: (0 = zginął, 1 = przeżył)

### 📚 Fabuła:

Jesteś analitykiem danych historycznych, zatrudnionym przez zespół badawczy zajmujący się katastrofą Titanica.

Celem Twojej pracy jest zbudowanie modelu predykcyjnego, który określi prawdopodobieństwo przeżycia pasażera na podstawie informacji takich jak wiek, płeć, klasa podróży czy liczba członków rodziny na pokładzie.

Model ma pomóc lepiej zrozumieć, jakie czynniki wpływały na szanse przeżycia i wesprzeć edukację, badania nad etyką ewakuacji oraz przyszłe procedury bezpieczeństwa na morzu.

### 🎯 Zadanie:
Jako analitycy danych, macie za zadanie stworzyć najlepszy możliwy model regresji logistycznej, który przewiduje szanse przeżycia pasażera Titanica.

#### 🔍 Waszym zadaniem jest:
1. Wybór dowolnych cech wejściowych z dostępnego zbioru danych (np. sex, pclass, age, fare, sibsp, parch, itd.)

2. (*Opcjonalne*) Przygotowanie danych – zadbanie o brakujące wartości, standaryzację/liczby binarne itp.

3. Zbudowanie własnego modelu regresji logistycznej:

  - na początek bez optymalizacji wag – ręcznie ustawcie współczynniki beta suwakami

4. Eksperymentowanie: sprawdźcie, jak różne kombinacje cech wpływają na dokładność modelu.

5. Zidentyfikujcie, które cechy mają największy wpływ na przeżycie.

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider

# Załaduj dataset Titanic z seaborn
df = sns.load_dataset('titanic')

# Zobacz pierwsze wiersze
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [ ]:
# Wybierz cechy
X = df[['age', 'fare']].values
y = df['survived'].values


# Dodaj kolumnę z jedynkami (bias)
X_aug = np.c_[np.ones(X.shape[0]), X]

# Sigmoid
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

# Funkcja do rysowania
def plot_decision_boundary(beta_0, beta_1, beta_2):
    weights = np.array([beta_0, beta_1, beta_2])
    z = X_aug @ weights
    probs = sigmoid(z)
    preds = (probs >= 0.5).astype(int)
    acc = np.mean(preds == y)
    colors = np.where(preds == y, 'green', 'red')

    plt.figure(figsize=(8,6))
    plt.scatter(X[:, 0], X[:, 1], c=colors, edgecolor='k', alpha=0.7)
    plt.xlabel('Wiek')
    plt.ylabel('Cena biletu')
    plt.title(f'Predykcja przeżycia – trafność: {acc*100:.1f}%')
    plt.grid(True)
    plt.show()

# Interaktywny widget
interact(
    plot_decision_boundary,
    beta_0=FloatSlider(value=0, min=-10, max=10, step=0.5, description='β₀ (bias)'),
    beta_1=FloatSlider(value=0, min=-5, max=5, step=0.1, description='β₁ (age)'),
    beta_2=FloatSlider(value=0, min=-5, max=5, step=0.1, description='β₂ (fare)'),
);

interactive(children=(FloatSlider(value=0.0, description='β₀ (bias)', max=10.0, min=-10.0, step=0.5), FloatSli…